In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
# PROJECT_ROOT_DIR = "."
# CHAPTER_ID = "deep"

# def save_fig(fig_id, tight_layout=True):
#     path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
#     print("Saving figure", fig_id)
#     if tight_layout:
#         plt.tight_layout()
#     plt.savefig(path, format='png', dpi=300)

深层神经网络的问题：
* 首先，梯度消失/爆炸问题
* 庞大网络，训练慢
* 参数越多，越容易过拟合

在本章，我们会回顾每一个问题，并且介绍科学解决方案。我们会从梯度消失问题开始，探索这类问题目前最流行的解决方案。接着会研究一下相较平坦梯度下降能够在训练大模型时做到明显提速的各种优化器。最后，我们会浏览一下几个流行的针对大型神经网络的正则化技术。

有了这些工具，我们就能训练比较深的网络：欢迎来到深度学习！

# 梯度消失/爆炸问题

我们可以通过反向传播算法来从输出层反向作用到输入层，传播梯度误差。

不幸的是，梯度经常会随着算法进展到更低层次时变得越来越小。导致的结果是，梯度下降在更低层网络连接权值更新方面基本没有改变，而且训练不会收敛到好的结果，这就是梯度消失问题。

在一些例子中会发生相反的现象，梯度会越来越大，导致很多层的权值疯狂增大，使得算法发散，这就是梯度爆炸，经常出现在循环神经网络中。

In [2]:
def logit(z):
    return 1 / (1 + np.exp(-z))

In [13]:
z = np.linspace(-5, 5, 200)

plt.plot([-5, 5], [0, 0], 'k-') # 从 (-5,0) -> (5,0)
plt.plot([-5, 5], [1, 1], 'k--')
plt.plot([0, 0], [-0.2, 1.2], 'k-')
plt.plot([-5, 5], [-3/4, 7/4], 'g--')
plt.plot(z, logit(z), "b-", linewidth=2)
props = dict(facecolor='black', shrink=0.1)
plt.annotate('Saturating', xytext=(3.5, 0.7), xy=(5, 1), arrowprops=props, fontsize=14, ha="center")
plt.annotate('Saturating', xytext=(-3.5, 0.3), xy=(-5, 0), arrowprops=props, fontsize=14, ha="center")
plt.annotate('Linear', xytext=(2, 0.2), xy=(0, 0.5), arrowprops=props, fontsize=14, ha="center")
plt.grid(True)
plt.title("Sigmoid activation function", fontsize=14)
plt.axis([-5, 5, -0.2, 1.2])

plt.show()

## Xavier初始化和He初始化

* Logistic uniform: $ r = \sqrt{\dfrac{6}{n_\text{inputs} + n_\text{outputs}}} $
* Logistic normal: $ \sigma = \sqrt{\dfrac{2}{n_\text{inputs} + n_\text{outputs}}} $
* Hyperbolic tangent uniform: $ r = 4 \sqrt{\dfrac{6}{n_\text{inputs} + n_\text{outputs}}} $
* Hyperbolic tangent normal: $ \sigma = 4 \sqrt{\dfrac{2}{n_\text{inputs} + n_\text{outputs}}} $
* ReLU (and its variants) uniform: $ r = \sqrt{2} \sqrt{\dfrac{6}{n_\text{inputs} + n_\text{outputs}}} $
* ReLU (and its variants) normal: $ \sigma = \sqrt{2} \sqrt{\dfrac{2}{n_\text{inputs} + n_\text{outputs}}} $

In [2]:
import tensorflow as tf

In [15]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

In [16]:
he_init = tf.variance_scaling_initializer()
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
                          kernel_initializer=he_init, name="hidden1")

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


## 非饱和激活函数

Clorot和Bengio在2010年的论文中提出一个观点，梯度消失/爆炸问题一部分的原因是选错了激活函数。人类根据仿生，假定Sigmoid函数是一个绝佳的选择，然后现在看来，其他很多激活函数表现更好，尤其是relu函数，出现这种结果的原因最主要是因为它并不稀释正值(同时运算速度很快)

然而，Relu会出现dying Relu问题：在训练过程中，一些神经元实际上已经死了，即它们只会输出0.

其改进为leaky Relu，给负值添加一个很小的坡度，这会使该节点进入昏迷期，但最后还是有机会醒过来，有论文表明，leaky Relu总是优于Relu。

然后是PRelu，将负值坡度作为训练参数。

最后2015年有人提出新的激活函数ELU，在测试中，这总是优于前面所有的Relu的变种：训练时间减小，测试集表现更好。ELU和其他变种很像，只是有几个明显的不同。
* 第一，负值输出接近0，缓和梯度消失，超参数为当z很大时，ELU函数接近的值，一般为1.
* 第二，对于z<0有非0的梯度，避免节点死亡
* 整体平滑，提高梯度下降

ELU激活函数的缺陷是计算速度慢，但是在训练过程中收敛速度快。

2017年，又有人提出了SELU函数，在训练过程中，利用selu激活函数和lecun初始化，由一堆密集层组成的神经网络会自我规范化：在训练过程中，每一层的输出往往保持相同的均值和方差，从而解决了梯度消失/爆炸问题。因此，对于这样的神经网络，这个激活函数比其他的激活函数表现得非常好，所以你应该尝试一下。不幸的是，selu激活函数的自规范化特性很容易被破坏：不能使用1或2正则化、正则辍学、最大范数、跳跃连接或其他非顺序拓扑（因此递归神经网络不会自规范化）。然而，在实践中，它与连续cnn非常好地配合使用。如果你打破了自我规范化，selu不一定会优于其他激活函数。

默认的SELU有`scale`和`alpha`两个超参数,elu超参数（scale和alpha）的调整方式是，每个神经元的平均输出保持接近0，标准偏差保持接近1（假设输入也用平均值0和标准偏差1标准化）。使用此激活函数，即使是1000层深的神经网络，也会在所有层上保留大致的平均值0和标准差1，从而避免爆炸/消失梯度问题：

In [17]:
def leaky_relu(z, alpha=0.01):
    return np.maximum(alpha*z, z)

In [20]:
plt.plot(z, leaky_relu(z, 0.05), "b-", linewidth=2)
plt.plot([-5, 5], [0, 0], 'k-')
plt.plot([0, 0], [-0.5, 4.2], 'k-')
plt.grid(True)
props = dict(facecolor='black', shrink=0.1)
plt.annotate('Leak', xytext=(-3.5, 0.5), xy=(-5, -0.2), arrowprops=props, fontsize=14, ha="center")
plt.title("Leaky ReLU activation function", fontsize=14)
plt.axis([-5, 5, -0.5, 4.2])

plt.show()

In [21]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

In [22]:
def leaky_relu(z, name=None):
    return tf.maximum(0.01 * z, z, name=name)

hidden1 = tf.layers.dense(X, n_hidden1, activation=leaky_relu, name="hidden1")

使用Leaky relu来训练

In [23]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [24]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [25]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=leaky_relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=leaky_relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [26]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [27]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [28]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [29]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [31]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [32]:
n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 5 == 0:
            acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
            acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
            print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Batch accuracy: 0.86 Validation accuracy: 0.9048
5 Batch accuracy: 0.94 Validation accuracy: 0.9496
10 Batch accuracy: 0.92 Validation accuracy: 0.9654
15 Batch accuracy: 0.94 Validation accuracy: 0.9714
20 Batch accuracy: 1.0 Validation accuracy: 0.9762
25 Batch accuracy: 1.0 Validation accuracy: 0.9776
30 Batch accuracy: 0.98 Validation accuracy: 0.978
35 Batch accuracy: 1.0 Validation accuracy: 0.9786


ELU训练

In [73]:
def elu(z, alpha=1):
    return np.where(z < 0, alpha * (np.exp(z) - 1), z)

In [74]:
plt.plot(z, elu(z), "b-", linewidth=2)
plt.plot([-5, 5], [0, 0], 'k-')
plt.plot([-5, 5], [-1, -1], 'k--')
plt.plot([0, 0], [-2.2, 3.2], 'k-')
plt.grid(True)
plt.title(r"ELU activation function ($\alpha=1$)", fontsize=14)
plt.axis([-5, 5, -2.2, 3.2])

plt.show()

In [58]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [59]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [61]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.elu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.elu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [62]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [63]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [64]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [65]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [67]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [68]:
n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 5 == 0:
            acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
            acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
            print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Batch accuracy: 0.88 Validation accuracy: 0.9036
5 Batch accuracy: 0.94 Validation accuracy: 0.9338
10 Batch accuracy: 0.9 Validation accuracy: 0.9532
15 Batch accuracy: 0.9 Validation accuracy: 0.9622
20 Batch accuracy: 1.0 Validation accuracy: 0.9656
25 Batch accuracy: 1.0 Validation accuracy: 0.9698
30 Batch accuracy: 0.98 Validation accuracy: 0.9698
35 Batch accuracy: 0.98 Validation accuracy: 0.9726


SELU

In [78]:
from scipy.special import erfc

# alpha and scale to self normalize with mean 0 and standard deviation 1
# (see equation 14 in the paper):
alpha_0_1 = -np.sqrt(2 / np.pi) / (erfc(1/np.sqrt(2)) * np.exp(1/2) - 1)
scale_0_1 = (1 - erfc(1 / np.sqrt(2)) * np.sqrt(np.e)) * np.sqrt(2 * np.pi) * (2 * erfc(np.sqrt(2))*np.e**2 + np.pi*erfc(1/np.sqrt(2))**2*np.e - 2*(2+np.pi)*erfc(1/np.sqrt(2))*np.sqrt(np.e)+np.pi+2)**(-1/2)

In [79]:
def selu(z, scale=scale_0_1, alpha=alpha_0_1):
    return scale * elu(z, alpha)

In [80]:
z = np.linspace(-5, 5, 200)
plt.plot(z, selu(z), "b-", linewidth=2)
plt.plot([-5, 5], [0, 0], 'k-')
plt.plot([-5, 5], [-1.758, -1.758], 'k--')
plt.plot([0, 0], [-2.2, 3.2], 'k-')
plt.grid(True)
plt.title(r"SELU activation function", fontsize=14)
plt.axis([-5, 5, -2.2, 3.2])

plt.show()

In [81]:
np.random.seed(42)
Z = np.random.normal(size=(500, 100)) # standardized inputs
for layer in range(1000):
    W = np.random.normal(size=(100, 100), scale=np.sqrt(1 / 100)) # LeCun initialization
    Z = selu(np.dot(Z, W))
    means = np.mean(Z, axis=0).mean()
    stds = np.std(Z, axis=0).mean()
    if layer % 100 == 0:
        print("Layer {}: mean {:.2f}, std deviation {:.2f}".format(layer, means, stds))

Layer 0: mean -0.00, std deviation 1.00
Layer 100: mean 0.02, std deviation 0.96
Layer 200: mean 0.01, std deviation 0.90
Layer 300: mean -0.02, std deviation 0.92
Layer 400: mean 0.05, std deviation 0.89
Layer 500: mean 0.01, std deviation 0.93
Layer 600: mean 0.02, std deviation 0.92
Layer 700: mean -0.02, std deviation 0.90
Layer 800: mean 0.05, std deviation 0.83
Layer 900: mean 0.02, std deviation 1.00


In [82]:
def selu(z, scale=alpha_0_1, alpha=scale_0_1):
    return scale * tf.where(z >= 0.0, z, alpha * tf.nn.elu(z))

In [85]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=selu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=selu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden2, activation=selu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden2, activation=selu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden2, activation=selu, name="hidden5")
    hidden6 = tf.layers.dense(hidden5, n_hidden2, activation=selu, name="hidden6")
    hidden7 = tf.layers.dense(hidden6, n_hidden2, activation=selu, name="hidden7")
    hidden8 = tf.layers.dense(hidden7, n_hidden2, activation=selu, name="hidden8")
    hidden9 = tf.layers.dense(hidden8, n_hidden2, activation=selu, name="hidden9")
    hidden10 = tf.layers.dense(hidden9, n_hidden2, activation=selu, name="hidden10")
    hidden11 = tf.layers.dense(hidden10, n_hidden2, activation=selu, name="hidden11")
    logits = tf.layers.dense(hidden11, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()
n_epochs = 40
batch_size = 50

将数据缩放到以0为均值，1为方差

In [86]:
means = X_train.mean(axis=0, keepdims=True)
stds = X_train.std(axis=0, keepdims=True) + 1e-10
X_val_scaled = (X_valid - means) / stds

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            X_batch_scaled = (X_batch - means) / stds
            sess.run(training_op, feed_dict={X: X_batch_scaled, y: y_batch})
        if epoch % 5 == 0:
            acc_batch = accuracy.eval(feed_dict={X: X_batch_scaled, y: y_batch})
            acc_valid = accuracy.eval(feed_dict={X: X_val_scaled, y: y_valid})
            print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

    save_path = saver.save(sess, "./my_model_final_selu.ckpt")

0 Batch accuracy: 0.96 Validation accuracy: 0.9332
5 Batch accuracy: 0.98 Validation accuracy: 0.9588
10 Batch accuracy: 1.0 Validation accuracy: 0.9626
15 Batch accuracy: 1.0 Validation accuracy: 0.962
20 Batch accuracy: 1.0 Validation accuracy: 0.9672
25 Batch accuracy: 1.0 Validation accuracy: 0.967
30 Batch accuracy: 1.0 Validation accuracy: 0.9674
35 Batch accuracy: 1.0 Validation accuracy: 0.9674


## 批量归一化

在激活函数之前，添加一个操作，简单零中心化和归一化输入，之后再通过每层的两个新参数(一个为了缩放，另一个为了移动)缩放和移动结果。

作者提出这个技术，考虑了饱和计划函数(比如tanh)和逻辑函数，网络对于权重参数不那么敏感,而且可以使用更高的学习率学习，但是批标准化也给模型增加了复杂度，此外模型网络的预测速度降低了。

一般而言，当梯度下降在每一层搜索最佳缩放和偏移量时，训练速度会非常慢，一旦找到，训练速度会迅速提升。

In [87]:
reset_graph()

import tensorflow as tf

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

training = tf.placeholder_with_default(False, shape=(), name='training')

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = tf.layers.batch_normalization(logits_before_bn, training=training,
                                       momentum=0.9)

Instructions for updating:
Use keras.layers.batch_normalization instead.


In [88]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
training = tf.placeholder_with_default(False, shape=(), name='training')

In [89]:
from functools import partial
# 避免重复使用相同的超参数
my_batch_norm_layer = partial(tf.layers.batch_normalization,
                              training=training, momentum=0.9)

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = my_batch_norm_layer(hidden1)
bn1_act = tf.nn.elu(bn1)
hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = my_batch_norm_layer(hidden2)
bn2_act = tf.nn.elu(bn2)
logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = my_batch_norm_layer(logits_before_bn)

In [90]:
reset_graph()

batch_norm_momentum = 0.9

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")
training = tf.placeholder_with_default(False, shape=(), name='training')

with tf.name_scope("dnn"):
    he_init = tf.variance_scaling_initializer()

    my_batch_norm_layer = partial(
            tf.layers.batch_normalization,
            training=training,
            momentum=batch_norm_momentum)

    my_dense_layer = partial(
            tf.layers.dense,
            kernel_initializer=he_init)

    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    bn1 = tf.nn.elu(my_batch_norm_layer(hidden1))
    hidden2 = my_dense_layer(bn1, n_hidden2, name="hidden2")
    bn2 = tf.nn.elu(my_batch_norm_layer(hidden2))
    logits_before_bn = my_dense_layer(bn2, n_outputs, name="outputs")
    logits = my_batch_norm_layer(logits_before_bn)

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

Instructions for updating:
Use tf.cast instead.


In [91]:
n_epochs = 20
batch_size = 200

In [92]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training_op, extra_update_ops],
                     feed_dict={training: True, X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.8952
1 Validation accuracy: 0.9202
2 Validation accuracy: 0.9318
3 Validation accuracy: 0.9422
4 Validation accuracy: 0.9468
5 Validation accuracy: 0.954
6 Validation accuracy: 0.9568
7 Validation accuracy: 0.96
8 Validation accuracy: 0.962
9 Validation accuracy: 0.9638
10 Validation accuracy: 0.9662
11 Validation accuracy: 0.9682
12 Validation accuracy: 0.9672
13 Validation accuracy: 0.9696
14 Validation accuracy: 0.9706
15 Validation accuracy: 0.9704
16 Validation accuracy: 0.9718
17 Validation accuracy: 0.9726
18 Validation accuracy: 0.9738
19 Validation accuracy: 0.9742


## 梯度裁剪

一个流行的减轻梯度爆炸问题的技术是在反向传播过程中，简单的裁剪梯度，从而保证不会超过这个阈值，对于循环神经网络非常有效。一般而言，大家倾向于批标准化，但是了解梯度裁剪还是很有必要的

In [93]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.relu, name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [94]:
learning_rate = 0.01

In [96]:
threshold = 1.0

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var)
              for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)

In [97]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [98]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [99]:
n_epochs = 20
batch_size = 200

In [100]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.288
1 Validation accuracy: 0.794
2 Validation accuracy: 0.8796
3 Validation accuracy: 0.9058
4 Validation accuracy: 0.9164
5 Validation accuracy: 0.9218
6 Validation accuracy: 0.929
7 Validation accuracy: 0.9356
8 Validation accuracy: 0.938
9 Validation accuracy: 0.9416
10 Validation accuracy: 0.9458
11 Validation accuracy: 0.9472
12 Validation accuracy: 0.9476
13 Validation accuracy: 0.9534
14 Validation accuracy: 0.9568
15 Validation accuracy: 0.9566
16 Validation accuracy: 0.9578
17 Validation accuracy: 0.959
18 Validation accuracy: 0.962
19 Validation accuracy: 0.9614


# 重用预训练图层

从头开始训练非常庞大的DNN并不明智，我们可以重用能处理相似问题的神经网络的底层网络，这就是迁移学习。

## 重用TensorFlow模型

In [103]:
reset_graph()

In [104]:
saver = tf.train.import_meta_graph("./my_model_final.ckpt.meta")

In [105]:
for op in tf.get_default_graph().get_operations():
    print(op.name)

X
y
hidden1/kernel/Initializer/random_uniform/shape
hidden1/kernel/Initializer/random_uniform/min
hidden1/kernel/Initializer/random_uniform/max
hidden1/kernel/Initializer/random_uniform/RandomUniform
hidden1/kernel/Initializer/random_uniform/sub
hidden1/kernel/Initializer/random_uniform/mul
hidden1/kernel/Initializer/random_uniform
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/zeros
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
dnn/hidden1/MatMul
dnn/hidden1/BiasAdd
dnn/hidden1/Relu
hidden2/kernel/Initializer/random_uniform/shape
hidden2/kernel/Initializer/random_uniform/min
hidden2/kernel/Initializer/random_uniform/max
hidden2/kernel/Initializer/random_uniform/RandomUniform
hidden2/kernel/Initializer/random_uniform/sub
hidden2/kernel/Initializer/random_uniform/mul
hidden2/kernel/Initializer/random_uniform
hidden2/kernel
hidden2/kernel/Assign
hidden2/kernel/read
hidden2/bias/Initializer/zeros
hidden2/bias
hidden2/bias/Assign
hidden2/bias/read
dn

In [ ]:
# from tensorflow_graph_in_jupyter import show_graph
# show_graph(tf.get_default_graph())

对之前的模型做进一步的包裹输入参数和一些计算节点

In [116]:
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

accuracy = tf.get_default_graph().get_tensor_by_name("eval/accuracy:0")

training_op = tf.get_default_graph().get_operation_by_name("GradientDescent")

In [117]:
for op in (X, y, accuracy, training_op):
    tf.add_to_collection("my_important_ops", op)

In [118]:
X, y, accuracy, training_op = tf.get_collection("my_important_ops")

In [119]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    # continue training the model...

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [120]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    print(tf.get_collection("my_important_ops"))
    X, y, accuracy, training_op = tf.get_collection("my_important_ops")
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")    

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
[<tf.Tensor 'X:0' shape=(?, 784) dtype=float32>, <tf.Tensor 'y:0' shape=<unknown> dtype=int32>, <tf.Tensor 'eval/accuracy:0' shape=() dtype=float32>, <tf.Operation 'GradientDescent' type=NoOp>]
0 Validation accuracy: 0.9636
1 Validation accuracy: 0.9646
2 Validation accuracy: 0.9648
3 Validation accuracy: 0.9644
4 Validation accuracy: 0.9666
5 Validation accuracy: 0.967
6 Validation accuracy: 0.9682
7 Validation accuracy: 0.9668
8 Validation accuracy: 0.966
9 Validation accuracy: 0.9712
10 Validation accuracy: 0.9676
11 Validation accuracy: 0.9704
12 Validation accuracy: 0.971
13 Validation accuracy: 0.9708
14 Validation accuracy: 0.9692
15 Validation accuracy: 0.97
16 Validation accuracy: 0.9712
17 Validation accuracy: 0.9688
18 Validation accuracy: 0.9726
19 Validation accuracy: 0.971


如果可以接触源码，就不需要`import_meta_graph()`

In [112]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.relu, name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

learning_rate = 0.01
threshold = 1.0

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var)
              for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)

saver = tf.train.Saver()

In [113]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")    

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 Validation accuracy: 0.9642
1 Validation accuracy: 0.9628
2 Validation accuracy: 0.9654
3 Validation accuracy: 0.9654
4 Validation accuracy: 0.9644
5 Validation accuracy: 0.9648
6 Validation accuracy: 0.9686
7 Validation accuracy: 0.9684
8 Validation accuracy: 0.968
9 Validation accuracy: 0.969
10 Validation accuracy: 0.9702
11 Validation accuracy: 0.971
12 Validation accuracy: 0.9668
13 Validation accuracy: 0.9702
14 Validation accuracy: 0.971
15 Validation accuracy: 0.9724
16 Validation accuracy: 0.9718
17 Validation accuracy: 0.971
18 Validation accuracy: 0.9712
19 Validation accuracy: 0.9712


如果可以接触源码，也可以使用部分和丢弃一些

In [121]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 # reused
n_hidden2 = 50  # reused
n_hidden3 = 50  # reused
n_hidden4 = 20  # new!
n_outputs = 10  # new!

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")       # reused
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2") # reused
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3") # reused
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4") # new!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs")                         # new!

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [122]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # regular expression
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):                                            # not shown in the book
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size): # not shown
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})        # not shown
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})     # not shown
        print(epoch, "Validation accuracy:", accuracy_val)                   # not shown

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 Validation accuracy: 0.9028
1 Validation accuracy: 0.9336
2 Validation accuracy: 0.943
3 Validation accuracy: 0.947
4 Validation accuracy: 0.9522
5 Validation accuracy: 0.9534
6 Validation accuracy: 0.9556
7 Validation accuracy: 0.9588
8 Validation accuracy: 0.9588
9 Validation accuracy: 0.9604
10 Validation accuracy: 0.9626
11 Validation accuracy: 0.962
12 Validation accuracy: 0.9642
13 Validation accuracy: 0.9662
14 Validation accuracy: 0.9664
15 Validation accuracy: 0.966
16 Validation accuracy: 0.9672
17 Validation accuracy: 0.9676
18 Validation accuracy: 0.9684
19 Validation accuracy: 0.9674


## 重用其他模型的框架

In [123]:
reset_graph()

n_inputs = 2
n_hidden1 = 3

In [124]:
original_w = [[1., 2., 3.], [4., 5., 6.]] # Load the weights from the other framework
original_b = [7., 8., 9.]                 # Load the biases from the other framework

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
# [...] Build the rest of the model

# Get a handle on the assignment nodes for the hidden1 variables
graph = tf.get_default_graph()
assign_kernel = graph.get_operation_by_name("hidden1/kernel/Assign")
assign_bias = graph.get_operation_by_name("hidden1/bias/Assign")
init_kernel = assign_kernel.inputs[1]
init_bias = assign_bias.inputs[1]

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init, feed_dict={init_kernel: original_w, init_bias: original_b})
    # [...] Train the model on your new task
    print(hidden1.eval(feed_dict={X: [[10.0, 11.0]]}))  # not shown in the book

[[ 61.  83. 105.]]


In [125]:
reset_graph()

n_inputs = 2
n_hidden1 = 3

original_w = [[1., 2., 3.], [4., 5., 6.]] # Load the weights from the other framework
original_b = [7., 8., 9.]                 # Load the biases from the other framework

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
# [...] Build the rest of the model

# Get a handle on the variables of layer hidden1
with tf.variable_scope("", default_name="", reuse=True):  # root scope
    hidden1_weights = tf.get_variable("hidden1/kernel")
    hidden1_biases = tf.get_variable("hidden1/bias")

# Create dedicated placeholders and assignment nodes
original_weights = tf.placeholder(tf.float32, shape=(n_inputs, n_hidden1))
original_biases = tf.placeholder(tf.float32, shape=n_hidden1)
assign_hidden1_weights = tf.assign(hidden1_weights, original_weights)
assign_hidden1_biases = tf.assign(hidden1_biases, original_biases)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    sess.run(assign_hidden1_weights, feed_dict={original_weights: original_w})
    sess.run(assign_hidden1_biases, feed_dict={original_biases: original_b})
    # [...] Train the model on your new task
    print(hidden1.eval(feed_dict={X: [[10.0, 11.0]]}))

[[ 61.  83. 105.]]


In [126]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="hidden1")

[<tf.Variable 'hidden1/kernel:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'hidden1/bias:0' shape=(3,) dtype=float32_ref>]

In [127]:
tf.get_default_graph().get_tensor_by_name("hidden1/kernel:0")

<tf.Tensor 'hidden1/kernel:0' shape=(2, 3) dtype=float32_ref>

In [128]:
tf.get_default_graph().get_tensor_by_name("hidden1/bias:0")

<tf.Tensor 'hidden1/bias:0' shape=(3,) dtype=float32_ref>

## 冻结低层

第一个DNN的底层可能已经学会了检测图像中低级特征，这对于图像分类任务都是有用的，因此我们可以直接重用这些特征。通常来说，底层权重被固定，那么高层的权重就比较容易训练。

In [129]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 # reused
n_hidden2 = 50  # reused
n_hidden3 = 50  # reused
n_hidden4 = 20  # new!
n_outputs = 10  # new!

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")       # reused
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2") # reused
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3") # reused
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4") # new!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs")                         # new!

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [130]:
# 第一种方法，将需要训练的参数进行梯度下降
with tf.name_scope("train"):                                         # not shown in the book
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)     # not shown
    train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                   scope="hidden[34]|outputs")
    training_op = optimizer.minimize(loss, var_list=train_vars)

In [131]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # regular expression
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 Validation accuracy: 0.8962
1 Validation accuracy: 0.9292
2 Validation accuracy: 0.94
3 Validation accuracy: 0.9438
4 Validation accuracy: 0.9482
5 Validation accuracy: 0.9506
6 Validation accuracy: 0.951
7 Validation accuracy: 0.9538
8 Validation accuracy: 0.9552
9 Validation accuracy: 0.9566
10 Validation accuracy: 0.9564
11 Validation accuracy: 0.9568
12 Validation accuracy: 0.957
13 Validation accuracy: 0.9576
14 Validation accuracy: 0.9592
15 Validation accuracy: 0.9576
16 Validation accuracy: 0.9572
17 Validation accuracy: 0.9602
18 Validation accuracy: 0.959
19 Validation accuracy: 0.9604


In [136]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 # reused
n_hidden2 = 50  # reused
n_hidden3 = 50  # reused
n_hidden4 = 20  # new!
n_outputs = 10  # new!

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [137]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
                              name="hidden1") # reused frozen
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu,
                              name="hidden2") # reused frozen
    hidden2_stop = tf.stop_gradient(hidden2) #对反向传播做截断
    hidden3 = tf.layers.dense(hidden2_stop, n_hidden3, activation=tf.nn.relu,
                              name="hidden3") # reused, not frozen
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu,
                              name="hidden4") # new!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs") # new!

In [138]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [139]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # regular expression
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 Validation accuracy: 0.9014
1 Validation accuracy: 0.931
2 Validation accuracy: 0.9434
3 Validation accuracy: 0.9474
4 Validation accuracy: 0.9516
5 Validation accuracy: 0.952
6 Validation accuracy: 0.9524
7 Validation accuracy: 0.9556
8 Validation accuracy: 0.9554
9 Validation accuracy: 0.956
10 Validation accuracy: 0.957
11 Validation accuracy: 0.955
12 Validation accuracy: 0.9572
13 Validation accuracy: 0.9578
14 Validation accuracy: 0.9578
15 Validation accuracy: 0.9572
16 Validation accuracy: 0.9564
17 Validation accuracy: 0.9576
18 Validation accuracy: 0.9592
19 Validation accuracy: 0.9578


## 缓存冻结层

基于上述的冻结层，我们已经将原有的模型的第一层与第二层进行了冻结，那么表示他的权重并不发生变化。如果现在训练新的网络数据仍然从第一层开始，这样会增加很多计算的开销，由于前两层的权重已经不会改变了，这时明智的做法是将第二层的输出作为新的网络的输入。

In [140]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 # reused
n_hidden2 = 50  # reused
n_hidden3 = 50  # reused
n_hidden4 = 20  # new!
n_outputs = 10  # new!

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
                              name="hidden1") # reused frozen
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu,
                              name="hidden2") # reused frozen & cached
    hidden2_stop = tf.stop_gradient(hidden2)
    hidden3 = tf.layers.dense(hidden2_stop, n_hidden3, activation=tf.nn.relu,
                              name="hidden3") # reused, not frozen
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu,
                              name="hidden4") # new!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs") # new!

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [141]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # regular expression
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [142]:
import numpy as np

n_batches = len(X_train) // batch_size

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")
    
    h2_cache = sess.run(hidden2, feed_dict={X: X_train})
    h2_cache_valid = sess.run(hidden2, feed_dict={X: X_valid}) # not shown in the book

    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(len(X_train))
        hidden2_batches = np.array_split(h2_cache[shuffled_idx], n_batches)
        y_batches = np.array_split(y_train[shuffled_idx], n_batches)
        for hidden2_batch, y_batch in zip(hidden2_batches, y_batches):
            sess.run(training_op, feed_dict={hidden2:hidden2_batch, y:y_batch})

        accuracy_val = accuracy.eval(feed_dict={hidden2: h2_cache_valid, # not shown
                                                y: y_valid})             # not shown
        print(epoch, "Validation accuracy:", accuracy_val)               # not shown

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 Validation accuracy: 0.9014
1 Validation accuracy: 0.931
2 Validation accuracy: 0.9434
3 Validation accuracy: 0.9474
4 Validation accuracy: 0.9516
5 Validation accuracy: 0.952
6 Validation accuracy: 0.9524
7 Validation accuracy: 0.9556
8 Validation accuracy: 0.9554
9 Validation accuracy: 0.956
10 Validation accuracy: 0.957
11 Validation accuracy: 0.955
12 Validation accuracy: 0.9572
13 Validation accuracy: 0.9578
14 Validation accuracy: 0.9578
15 Validation accuracy: 0.9572
16 Validation accuracy: 0.9564
17 Validation accuracy: 0.9576
18 Validation accuracy: 0.9592
19 Validation accuracy: 0.9578


## 调整、丢弃或替换高层

原始模型的输出层经常被丢弃，还有高层特征。在新任务采用原始模型进行训练时，一般冻结全部，然后从上往下逐层解冻或者丢弃一些，然后尝试效果，也可以替换和添加，然后尝试效果。

## 模型动物园

我们可以用我们自己的模型，也可以去 [Model Zoo](https://github.com/tensorflow/models)里面寻找

## 无监督的预训练

如果我们要完成没有太多标记训练数据的复杂任务，可以先使用无监督学习(RBM或自动编码器)，每一层都基于提前训练好的图层的输出进行训练，然后使用监督学习微调网络。

## 辅助任务中的预训练

最后一个选择是在辅助任务中训练第一个简单的神经网络，重用该底层，继续训练。

# 快速优化器

一般而言，无脑用Adam优化就好。

## Momentum优化

简单而言就是梯度被当做加速度

In [143]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9)

## Nesterov梯度加速

简单而言，就是不是计算当前点的加速度，而是未来的加速度

In [ ]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9, use_nesterov=True)

## AdaGrad

可以衰减学习率，通过累计学习率

In [ ]:
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)

## RMSProp

解决Ada降速太快，无法收敛到全局最优的问题，仅累计最近迭代的梯度。

In [ ]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate,
                                      momentum=0.9, decay=0.9, epsilon=1e-10)

## Adam优化

集合动量优化和RMSProp的优点，跟踪过去梯度的指数衰减平均值，同时也跟踪过去梯度平方的指数衰减平均律。

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

## 学习速率调优

以一个高学习速率开始，然后一旦停止快速过程就降低学习率，会比最优固定学习速率更快的方案。这些策略很多：
1. 预定分段常数学习率
1. 性能调度，每一定步骤对错误进行验证，然后决定是否衰减学习率
1. 指数调度，学习率随迭代次数指数衰减，调节超参数 比率和迭代次数
1. 功率调度 和指数调度很像，但是学习速率降低很慢

In [145]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [146]:
with tf.name_scope("train"):       # not shown in the book
    initial_learning_rate = 0.1
    decay_steps = 10000
    decay_rate = 1/10
    global_step = tf.Variable(0, trainable=False, name="global_step")
    # 指数衰减学习率
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step,
                                               decay_steps, decay_rate)
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss, global_step=global_step)

In [147]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [148]:
n_epochs = 5
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.9578
1 Validation accuracy: 0.9722
2 Validation accuracy: 0.9778
3 Validation accuracy: 0.98
4 Validation accuracy: 0.9826


# 通过正则化避免过度拟合

主要有如下几种方法:
* 提早停止
* l1和l2正则化
* dropout
* 最大范数正则化
* 数据扩充

## 提前停止

当验证集的性能开始下降时，停止。

## L1和L2正则化

l1就是损失函数加参数绝对值和，l2就是平方和

In [149]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    logits = tf.layers.dense(hidden1, n_outputs, name="outputs")

In [150]:
W1 = tf.get_default_graph().get_tensor_by_name("hidden1/kernel:0")
W2 = tf.get_default_graph().get_tensor_by_name("outputs/kernel:0")

scale = 0.001 # l1 regularization hyperparameter

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")
    reg_losses = tf.reduce_sum(tf.abs(W1)) + tf.reduce_sum(tf.abs(W2)) 
    loss = tf.add(base_loss, scale * reg_losses, name="loss") # 损失+参数绝对值和

In [151]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [152]:
n_epochs = 20
batch_size = 200

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.831
1 Validation accuracy: 0.871
2 Validation accuracy: 0.8838
3 Validation accuracy: 0.8934
4 Validation accuracy: 0.8966
5 Validation accuracy: 0.8988
6 Validation accuracy: 0.9016
7 Validation accuracy: 0.9044
8 Validation accuracy: 0.9058
9 Validation accuracy: 0.906
10 Validation accuracy: 0.9068
11 Validation accuracy: 0.9054
12 Validation accuracy: 0.907
13 Validation accuracy: 0.9084
14 Validation accuracy: 0.9088
15 Validation accuracy: 0.9064
16 Validation accuracy: 0.9066
17 Validation accuracy: 0.9066
18 Validation accuracy: 0.9066
19 Validation accuracy: 0.9052


更好的方法

In [153]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [154]:
scale = 0.001

In [155]:
my_dense_layer = partial(
    tf.layers.dense, activation=tf.nn.relu,
    kernel_regularizer=tf.contrib.layers.l1_regularizer(scale)) # 指定l1正则超参数

with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    hidden2 = my_dense_layer(hidden1, n_hidden2, name="hidden2")
    logits = my_dense_layer(hidden2, n_outputs, activation=None,
                            name="outputs")


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [156]:
with tf.name_scope("loss"):                                     # not shown in the book
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(  # not shown
        labels=y, logits=logits)                                # not shown
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")   # not shown
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES) # 获取l1超参数的值
    loss = tf.add_n([base_loss] + reg_losses, name="loss")     # 损失+参数绝对值和

In [157]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [158]:
n_epochs = 20
batch_size = 200

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.8274
1 Validation accuracy: 0.8766
2 Validation accuracy: 0.8952
3 Validation accuracy: 0.9016
4 Validation accuracy: 0.908
5 Validation accuracy: 0.9096
6 Validation accuracy: 0.9124
7 Validation accuracy: 0.9154
8 Validation accuracy: 0.9178
9 Validation accuracy: 0.919
10 Validation accuracy: 0.92
11 Validation accuracy: 0.9224
12 Validation accuracy: 0.9212
13 Validation accuracy: 0.9228
14 Validation accuracy: 0.9222
15 Validation accuracy: 0.9216
16 Validation accuracy: 0.9218
17 Validation accuracy: 0.9228
18 Validation accuracy: 0.9216
19 Validation accuracy: 0.9214


## Dropout

另一种最受欢迎的正则化技术是dropout，非常暴力，效果非常好，随机暂时丢弃一些节点。

In [159]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [160]:
training = tf.placeholder_with_default(False, shape=(), name='training')

dropout_rate = 0.5  # == 1 - keep_prob
X_drop = tf.layers.dropout(X, dropout_rate, training=training)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X_drop, n_hidden1, activation=tf.nn.relu,
                              name="hidden1")
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
    hidden2 = tf.layers.dense(hidden1_drop, n_hidden2, activation=tf.nn.relu,
                              name="hidden2")
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)
    logits = tf.layers.dense(hidden2_drop, n_outputs, name="outputs")

Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [161]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [162]:
n_epochs = 20
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.9264
1 Validation accuracy: 0.9464
2 Validation accuracy: 0.9518
3 Validation accuracy: 0.9554
4 Validation accuracy: 0.9586
5 Validation accuracy: 0.962
6 Validation accuracy: 0.9614
7 Validation accuracy: 0.9656
8 Validation accuracy: 0.9694
9 Validation accuracy: 0.9712
10 Validation accuracy: 0.9704
11 Validation accuracy: 0.9712
12 Validation accuracy: 0.9702
13 Validation accuracy: 0.9726
14 Validation accuracy: 0.9736
15 Validation accuracy: 0.971
16 Validation accuracy: 0.9716
17 Validation accuracy: 0.9742
18 Validation accuracy: 0.9732
19 Validation accuracy: 0.9744


## 最大范数正则化

对每一个节点的权重参数做限制

In [163]:
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

learning_rate = 0.01
momentum = 0.9

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [164]:
threshold = 1.0
weights = tf.get_default_graph().get_tensor_by_name("hidden1/kernel:0")
# 限制节点权重值
clipped_weights = tf.clip_by_norm(weights, clip_norm=threshold, axes=1)
clip_weights = tf.assign(weights, clipped_weights)

In [165]:
weights2 = tf.get_default_graph().get_tensor_by_name("hidden2/kernel:0")
# 限制节点权重值
clipped_weights2 = tf.clip_by_norm(weights2, clip_norm=threshold, axes=1)
clip_weights2 = tf.assign(weights2, clipped_weights2)

In [166]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [167]:
n_epochs = 20
batch_size = 50

In [168]:
with tf.Session() as sess:                                              # not shown in the book
    init.run()                                                          # not shown
    for epoch in range(n_epochs):                                       # not shown
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size): # not shown
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            clip_weights.eval()
            clip_weights2.eval()                                        # not shown
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})   # not shown
        print(epoch, "Validation accuracy:", acc_valid)                 # not shown

    save_path = saver.save(sess, "./my_model_final.ckpt")               # not shown

0 Validation accuracy: 0.9566
1 Validation accuracy: 0.9704
2 Validation accuracy: 0.9722
3 Validation accuracy: 0.9764
4 Validation accuracy: 0.9772
5 Validation accuracy: 0.9778
6 Validation accuracy: 0.9822
7 Validation accuracy: 0.9808
8 Validation accuracy: 0.9808
9 Validation accuracy: 0.9822
10 Validation accuracy: 0.983
11 Validation accuracy: 0.9842
12 Validation accuracy: 0.982
13 Validation accuracy: 0.984
14 Validation accuracy: 0.9844
15 Validation accuracy: 0.985
16 Validation accuracy: 0.9848
17 Validation accuracy: 0.9846
18 Validation accuracy: 0.985
19 Validation accuracy: 0.9852


更好的方法

In [169]:
def max_norm_regularizer(threshold, axes=1, name="max_norm",
                         collection="max_norm"):
    def max_norm(weights):
        clipped = tf.clip_by_norm(weights, clip_norm=threshold, axes=axes)
        clip_weights = tf.assign(weights, clipped, name=name)
        tf.add_to_collection(collection, clip_weights)
        return None # there is no regularization loss term
    return max_norm

In [170]:
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

learning_rate = 0.01
momentum = 0.9

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [171]:
max_norm_reg = max_norm_regularizer(threshold=1.0)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
                              kernel_regularizer=max_norm_reg, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu,
                              kernel_regularizer=max_norm_reg, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [172]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [173]:
n_epochs = 20
batch_size = 50

In [174]:
clip_all_weights = tf.get_collection("max_norm")

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            sess.run(clip_all_weights)
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid}) # not shown
        print(epoch, "Validation accuracy:", acc_valid)               # not shown

    save_path = saver.save(sess, "./my_model_final.ckpt")             # not shown

0 Validation accuracy: 0.9558
1 Validation accuracy: 0.9698
2 Validation accuracy: 0.973
3 Validation accuracy: 0.975
4 Validation accuracy: 0.9762
5 Validation accuracy: 0.9782
6 Validation accuracy: 0.9798
7 Validation accuracy: 0.9796
8 Validation accuracy: 0.9812
9 Validation accuracy: 0.9812
10 Validation accuracy: 0.9824
11 Validation accuracy: 0.9808
12 Validation accuracy: 0.9814
13 Validation accuracy: 0.9822
14 Validation accuracy: 0.9828
15 Validation accuracy: 0.9816
16 Validation accuracy: 0.9814
17 Validation accuracy: 0.9826
18 Validation accuracy: 0.9822
19 Validation accuracy: 0.9814


## 数据扩充

模型训练的不好，那一定是数据不够多，让我们来增强数据集。通过种种方式，对数据集进行变换，扩增数据集数量。

# 实用指南

本章介绍了很多技术，如果搞不清，可以看这个表

| First Header  | Second Header |
| ------------- | ------------- |
| 初始化  | He初始化  |
| 激活函数  | ELU  |
| 标准化  | 批标准化  |
| 正则化  | dropoout  |
| 优化器  | Adam  |
| 学习速率调度  | None  |


* 如果找不到良好的学习率，可以尝试指数学习衰减(原本收敛慢，现在想提高学习率，然后收敛快了，正确性降低)
* 训练集小，就扩充数据集
* 需要稀疏模型，就加l1正则化，并且训练之后选择权重为0.如果想要更加稀疏，就用FTRL代替Adam，搭配l1正则化
* 如果想要运行时快，可能需要丢弃批标准化，同时使用leaky relu代替ELU激活函数。

有了这些准则，就可以开始训练深度神经网络……很有耐心的情况下。

如果只有一台设备，那么训练可能需要等待几天甚至几个月。下一章，将会介绍怎么使用分布式TensorFlow在许多服务器和GPU上训练。

# 练习

## 8.深度学习

用每100个神经元5个隐藏层，使用He初始化和ELU激活函数构建一个DNN

In [4]:
he_init = tf.variance_scaling_initializer()

def dnn(inputs, n_hidden_layers=5, n_neurons=100, name=None,
        activation=tf.nn.elu, initializer=he_init):
    with tf.variable_scope(name, "dnn"):
        for layer in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs, n_neurons, activation=activation,
                                     kernel_initializer=initializer,
                                     name="hidden%d" % (layer + 1))
        return inputs

In [5]:
n_inputs = 28 * 28 # MNIST
n_outputs = 5

reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

dnn_outputs = dnn(X)

logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
Y_proba = tf.nn.softmax(logits, name="Y_proba")

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


使用adam优化器和提前停止，尝试在MNIST上训练0-4之间的数字，定期检查模型点，最后保存最后的模型

In [6]:
learning_rate = 0.01

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss, name="training_op")

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [7]:
X_train1 = X_train[y_train < 5]
y_train1 = y_train[y_train < 5]
X_valid1 = X_valid[y_valid < 5]
y_valid1 = y_valid[y_valid < 5]
X_test1 = X_test[y_test < 5]
y_test1 = y_test[y_test < 5]

In [8]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train1))
        for rnd_indices in np.array_split(rnd_idx, len(X_train1) // batch_size):
            X_batch, y_batch = X_train1[rnd_indices], y_train1[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid1, y: y_valid1})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./my_mnist_model_0_to_4.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    saver.restore(sess, "./my_mnist_model_0_to_4.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test1, y: y_test1})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

0	Validation loss: 0.170243	Best loss: 0.170243	Accuracy: 97.11%
1	Validation loss: 0.090956	Best loss: 0.090956	Accuracy: 97.50%
2	Validation loss: 1.207547	Best loss: 0.090956	Accuracy: 38.39%
3	Validation loss: 1.212253	Best loss: 0.090956	Accuracy: 39.05%
4	Validation loss: 0.850490	Best loss: 0.090956	Accuracy: 60.48%
5	Validation loss: 0.174748	Best loss: 0.090956	Accuracy: 96.68%
6	Validation loss: 0.351392	Best loss: 0.090956	Accuracy: 94.80%
7	Validation loss: 0.254603	Best loss: 0.090956	Accuracy: 95.07%
8	Validation loss: 0.203032	Best loss: 0.090956	Accuracy: 95.70%
9	Validation loss: 0.491665	Best loss: 0.090956	Accuracy: 86.90%
10	Validation loss: 1.289185	Best loss: 0.090956	Accuracy: 39.48%
11	Validation loss: 1.281717	Best loss: 0.090956	Accuracy: 42.38%
12	Validation loss: 1.134424	Best loss: 0.090956	Accuracy: 39.48%
13	Validation loss: 1.882940	Best loss: 0.090956	Accuracy: 91.20%
14	Validation loss: 0.298495	Best loss: 0.090956	Accuracy: 91.01%
15	Validation loss: 

定义一个DNN的sklearn的模型

In [9]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
    # 初始化超参数
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=20, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """
            Initialize the DNNClassifier by simply storing all the hyperparameters.
            n_hidden_layers=5,  网络层数
            n_neurons=100,  神经元个数
            optimizer_class=tf.train.AdamOptimizer 优化器,
            learning_rate=0.01,  学习率
            batch_size=20,  每一次训练批量
            activation=tf.nn.elu,  激活函数
            initializer=he_init, 参数初始化方式
            batch_norm_momentum=None,  是否批标准化
            dropout_rate=None,  是否dropout
            random_state=None 随机值
        """
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None
    # 定义模型
    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
#             是否是同dropout
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
#             是否批标准化
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs
#   构建计算图
    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        # 是否固定随机值
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)
#         输出参数
        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")
#         如果批标准化或者dropout，那么需要定义区分训练和测试的超参数
        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        dnn_outputs = self._dnn(X)

        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                  logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)
#       验证结果
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()
# 获取模型参数
    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}
# 给模型参数赋值
    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)
# 训练
    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        # 输入和输出数据的维度
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        # 将数据结果和标签对齐
        self.class_to_index_ = {label: index
                                for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        # needed in case of early stopping
        max_checks_without_progress = 20
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                 feed_dict={self._X: X_valid,
                                                            self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print("Early stopping!")
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                     feed_dict={self._X: X_batch,
                                                                self._y: y_batch})
                    print("{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_train, acc_train * 100))
            # If we used early stopping then rollback to the best model found
            if best_params:
                self._restore_model_params(best_params)
            return self

    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)

In [10]:
dnn_clf = DNNClassifier(random_state=42)
dnn_clf.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

0	Validation loss: 0.297182	Best loss: 0.297182	Accuracy: 87.06%
1	Validation loss: 0.094552	Best loss: 0.094552	Accuracy: 97.69%
2	Validation loss: 0.124668	Best loss: 0.094552	Accuracy: 97.85%
3	Validation loss: 0.155978	Best loss: 0.094552	Accuracy: 96.52%
4	Validation loss: 0.143323	Best loss: 0.094552	Accuracy: 97.73%
5	Validation loss: 0.106396	Best loss: 0.094552	Accuracy: 97.85%
6	Validation loss: 0.172787	Best loss: 0.094552	Accuracy: 96.33%
7	Validation loss: 0.195093	Best loss: 0.094552	Accuracy: 95.86%
8	Validation loss: 0.138498	Best loss: 0.094552	Accuracy: 97.77%
9	Validation loss: 0.149798	Best loss: 0.094552	Accuracy: 96.87%
10	Validation loss: 0.159956	Best loss: 0.094552	Accuracy: 97.15%
11	Validation loss: 0.127047	Best loss: 0.094552	Accuracy: 97.42%
12	Validation loss: 0.111114	Best loss: 0.094552	Accuracy: 98.12%
13	Validation loss: 0.251291	Best loss: 0.094552	Accuracy: 97.26%
14	Validation loss: 0.965917	Best loss: 0.094552	Accuracy: 98.08%
15	Validation loss: 

DNNClassifier(activation=<function elu at 0x0000013027896D08>,
              batch_norm_momentum=None, batch_size=20, dropout_rate=None,
              initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x0000013041242CC0>,
              learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
              optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
              random_state=42)

In [11]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9873516248297334

交叉验证搜索超参数

In [12]:
from sklearn.model_selection import RandomizedSearchCV

def leaky_relu(alpha=0.01):
    def parametrized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parametrized_leaky_relu

param_distribs = {
    "n_neurons": [10, 30, 50, 70, 90, 100, 120, 140, 160],
    "batch_size": [10, 50, 100, 500],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    # you could also try exploring different numbers of hidden layers, different optimizers, etc.
    #"n_hidden_layers": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    #"optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
}

rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
                                cv=3, random_state=42, verbose=2)
rnd_search.fit(X_train1, y_train1, X_valid=X_valid1, y_valid=y_valid1, n_epochs=1000)

# If you have Scikit-Learn 0.18 or earlier, you should upgrade, or use the fit_params argument:
# fit_params = dict(X_valid=X_valid1, y_valid=y_valid1, n_epochs=1000)
# rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
#                                 fit_params=fit_params, random_state=42, verbose=2)
# rnd_search.fit(X_train1, y_train1)


Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] n_neurons=10, learning_rate=0.05, batch_size=100, activation=<function elu at 0x0000013027896D08> 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0	Validation loss: 0.286659	Best loss: 0.286659	Accuracy: 93.24%
1	Validation loss: 0.123074	Best loss: 0.123074	Accuracy: 97.30%
2	Validation loss: 0.120591	Best loss: 0.120591	Accuracy: 97.07%
3	Validation loss: 0.127779	Best loss: 0.120591	Accuracy: 96.91%
4	Validation loss: 0.106498	Best loss: 0.106498	Accuracy: 97.19%
5	Validation loss: 0.132649	Best loss: 0.106498	Accuracy: 96.76%
6	Validation loss: 0.216658	Best loss: 0.106498	Accuracy: 94.18%
7	Validation loss: 0.118795	Best loss: 0.106498	Accuracy: 96.72%
8	Validation loss: 0.104600	Best loss: 0.104600	Accuracy: 97.42%
9	Validation loss: 0.127365	Best loss: 0.104600	Accuracy: 96.48%
10	Validation loss: 0.162556	Best loss: 0.104600	Accuracy: 96.72%
11	Validation loss: 0.156763	Best loss: 0.104600	Accuracy: 96.36%
12	Validation loss: 0.189163	Best loss: 0.104600	Accuracy: 95.97%
13	Validation loss: 1.732670	Best loss: 0.104600	Accuracy: 18.73%
14	Validation loss: 1.621623	Best loss: 0.104600	Accuracy: 20.91%
15	Validation loss: 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.6s remaining:    0.0s


0	Validation loss: 0.185568	Best loss: 0.185568	Accuracy: 95.74%
1	Validation loss: 0.139123	Best loss: 0.139123	Accuracy: 97.11%
2	Validation loss: 0.139343	Best loss: 0.139123	Accuracy: 96.64%
3	Validation loss: 0.187580	Best loss: 0.139123	Accuracy: 96.25%
4	Validation loss: 1.337599	Best loss: 0.139123	Accuracy: 35.89%
5	Validation loss: 1.226708	Best loss: 0.139123	Accuracy: 43.90%
6	Validation loss: 0.794379	Best loss: 0.139123	Accuracy: 63.02%
7	Validation loss: 0.927424	Best loss: 0.139123	Accuracy: 62.63%
8	Validation loss: 0.427080	Best loss: 0.139123	Accuracy: 82.64%
9	Validation loss: 0.378933	Best loss: 0.139123	Accuracy: 87.84%
10	Validation loss: 0.438344	Best loss: 0.139123	Accuracy: 87.84%
11	Validation loss: 0.302510	Best loss: 0.139123	Accuracy: 92.34%
12	Validation loss: 0.294068	Best loss: 0.139123	Accuracy: 92.61%
13	Validation loss: 0.305502	Best loss: 0.139123	Accuracy: 93.67%
14	Validation loss: 0.296342	Best loss: 0.139123	Accuracy: 93.16%
15	Validation loss: 

...

5	Validation loss: 1.615893	Best loss: 1.608690	Accuracy: 22.01%
6	Validation loss: 1.615373	Best loss: 1.608690	Accuracy: 22.01%
7	Validation loss: 1.615399	Best loss: 1.608690	Accuracy: 18.73%
8	Validation loss: 1.612343	Best loss: 1.608690	Accuracy: 22.01%
9	Validation loss: 1.610829	Best loss: 1.608690	Accuracy: 22.01%
10	Validation loss: 1.611877	Best loss: 1.608690	Accuracy: 22.01%
11	Validation loss: 1.612860	Best loss: 1.608690	Accuracy: 22.01%
12	Validation loss: 1.610815	Best loss: 1.608690	Accuracy: 20.91%
13	Validation loss: 1.620048	Best loss: 1.608690	Accuracy: 19.27%
14	Validation loss: 1.612821	Best loss: 1.608690	Accuracy: 22.01%
15	Validation loss: 1.615503	Best loss: 1.608690	Accuracy: 22.01%
16	Validation loss: 1.628744	Best loss: 1.608690	Accuracy: 18.73%
17	Validation loss: 1.625351	Best loss: 1.608690	Accuracy: 19.08%
18	Validation loss: 1.609345	Best loss: 1.608690	Accuracy: 20.91%
19	Validation loss: 1.613123	Best loss: 1.608690	Accuracy: 19.08%
20	Validation l

42	Validation loss: 28643652.000000	Best loss: 0.147067	Accuracy: 19.27%
43	Validation loss: 116534848.000000	Best loss: 0.147067	Accuracy: 19.27%
44	Validation loss: 1896918.750000	Best loss: 0.147067	Accuracy: 45.00%
45	Validation loss: 722278.750000	Best loss: 0.147067	Accuracy: 53.01%
46	Validation loss: 164039.187500	Best loss: 0.147067	Accuracy: 64.97%
47	Validation loss: 124646.179688	Best loss: 0.147067	Accuracy: 72.75%
48	Validation loss: 85472.867188	Best loss: 0.147067	Accuracy: 73.03%
49	Validation loss: 67209.898438	Best loss: 0.147067	Accuracy: 75.37%
50	Validation loss: 55072.550781	Best loss: 0.147067	Accuracy: 79.40%
51	Validation loss: 110608.601562	Best loss: 0.147067	Accuracy: 75.02%
52	Validation loss: 71109.570312	Best loss: 0.147067	Accuracy: 75.29%
53	Validation loss: 37764.347656	Best loss: 0.147067	Accuracy: 84.99%
54	Validation loss: 35765.789062	Best loss: 0.147067	Accuracy: 84.05%
55	Validation loss: 31033.107422	Best loss: 0.147067	Accuracy: 86.36%
56	Vali

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 62.2min finished


0	Validation loss: 0.069621	Best loss: 0.069621	Accuracy: 98.12%
1	Validation loss: 0.050563	Best loss: 0.050563	Accuracy: 98.40%
2	Validation loss: 0.042818	Best loss: 0.042818	Accuracy: 98.91%
3	Validation loss: 0.061125	Best loss: 0.042818	Accuracy: 98.16%
4	Validation loss: 0.044167	Best loss: 0.042818	Accuracy: 98.59%
5	Validation loss: 0.041982	Best loss: 0.041982	Accuracy: 98.87%
6	Validation loss: 0.036920	Best loss: 0.036920	Accuracy: 98.98%
7	Validation loss: 0.031289	Best loss: 0.031289	Accuracy: 99.18%
8	Validation loss: 0.051803	Best loss: 0.031289	Accuracy: 98.98%
9	Validation loss: 0.042392	Best loss: 0.031289	Accuracy: 98.94%
10	Validation loss: 0.045203	Best loss: 0.031289	Accuracy: 98.75%
11	Validation loss: 0.033064	Best loss: 0.031289	Accuracy: 98.94%
12	Validation loss: 0.045873	Best loss: 0.031289	Accuracy: 99.06%
13	Validation loss: 0.046464	Best loss: 0.031289	Accuracy: 98.79%
14	Validation loss: 0.056397	Best loss: 0.031289	Accuracy: 98.79%
15	Validation loss: 

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=DNNClassifier(activation=<function elu at 0x0000013027896D08>,
                                           batch_norm_momentum=None,
                                           batch_size=20, dropout_rate=None,
                                           initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x0000013041242CC0>,
                                           learning_rate=0.01,
                                           n_hidden_layers=5, n_neurons=100,
                                           optimizer_class=<class 'tensorf...
                                                       <function leaky_relu.<locals>.parametrized_leaky_relu at 0x0000013025918620>,
                                                       <function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000131B3399B70>],
                                        'batch_size': [10, 50, 100, 500],
           

In [13]:
rnd_search.best_params_

{'n_neurons': 90,
 'learning_rate': 0.01,
 'batch_size': 500,
 'activation': <function __main__.leaky_relu.<locals>.parametrized_leaky_relu(z, name=None)>}

In [14]:
y_pred = rnd_search.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9892975287020821

In [15]:
rnd_search.best_estimator_.save("./my_best_mnist_model_0_to_4")

In [16]:
dnn_clf = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=500, learning_rate=0.01,
                        n_neurons=140, random_state=42)
dnn_clf.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

0	Validation loss: 0.083501	Best loss: 0.083501	Accuracy: 97.54%
1	Validation loss: 0.051913	Best loss: 0.051913	Accuracy: 98.44%
2	Validation loss: 0.044070	Best loss: 0.044070	Accuracy: 98.51%
3	Validation loss: 0.055985	Best loss: 0.044070	Accuracy: 98.40%
4	Validation loss: 0.043384	Best loss: 0.043384	Accuracy: 98.51%
5	Validation loss: 0.054590	Best loss: 0.043384	Accuracy: 98.67%
6	Validation loss: 0.037550	Best loss: 0.037550	Accuracy: 99.02%
7	Validation loss: 0.058001	Best loss: 0.037550	Accuracy: 98.75%
8	Validation loss: 0.033041	Best loss: 0.033041	Accuracy: 98.91%
9	Validation loss: 0.060878	Best loss: 0.033041	Accuracy: 98.51%
10	Validation loss: 0.058504	Best loss: 0.033041	Accuracy: 98.79%
11	Validation loss: 0.079055	Best loss: 0.033041	Accuracy: 98.48%
12	Validation loss: 0.040542	Best loss: 0.033041	Accuracy: 99.06%
13	Validation loss: 0.049087	Best loss: 0.033041	Accuracy: 98.98%
14	Validation loss: 0.043807	Best loss: 0.033041	Accuracy: 98.91%
15	Validation loss: 

DNNClassifier(activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000131B6C846A8>,
              batch_norm_momentum=None, batch_size=500, dropout_rate=None,
              initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x0000013041242CC0>,
              learning_rate=0.01, n_hidden_layers=5, n_neurons=140,
              optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
              random_state=42)

In [17]:
y_pred = dnn_clf.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9920217941233703

尝试添加批标准化，再来

In [18]:
dnn_clf_bn = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=500, learning_rate=0.01,
                           n_neurons=90, random_state=42,
                           batch_norm_momentum=0.95)
dnn_clf_bn.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use tf.cast instead.
0	Validation loss: 0.044991	Best loss: 0.044991	Accuracy: 98.71%
1	Validation loss: 0.038801	Best loss: 0.038801	Accuracy: 98.67%
2	Validation loss: 0.038598	Best loss: 0.038598	Accuracy: 98.91%
3	Validation loss: 0.069569	Best loss: 0.038598	Accuracy: 97.58%
4	Validation loss: 0.056699	Best loss: 0.038598	Accuracy: 98.44%
5	Validation loss: 0.033293	Best loss: 0.033293	Accuracy: 98.79%
6	Validation loss: 0.035859	Best loss: 0.033293	Accuracy: 98.98%
7	Validation loss: 0.028731	Best loss: 0.028731	Accuracy: 99.26%
8	Validation loss: 0.038481	Best loss: 0.028731	Accuracy: 99.14%
9	Validation loss: 0.027905	Best loss: 0.027905	Accuracy: 99.34%
10	Validation loss: 0.051913	Best loss: 0.027905	Accuracy: 98.59%
11	Validation loss: 0.037121	Best loss: 0.027905	Accuracy: 98.91%
12	Validation loss: 0.036288	Best loss: 0.027905	Accuracy: 99.26%
13	Validation loss: 0.032389	Be

DNNClassifier(activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000131B899A620>,
              batch_norm_momentum=0.95, batch_size=500, dropout_rate=None,
              initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x0000013041242CC0>,
              learning_rate=0.01, n_hidden_layers=5, n_neurons=90,
              optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
              random_state=42)

In [19]:
y_pred = dnn_clf_bn.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9941622883829538

In [20]:
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_neurons": [10, 30, 50, 70, 90, 100, 120, 140, 160],
    "batch_size": [10, 50, 100, 500],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    # you could also try exploring different numbers of hidden layers, different optimizers, etc.
    #"n_hidden_layers": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    #"optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
    "batch_norm_momentum": [0.9, 0.95, 0.98, 0.99, 0.999],
}

rnd_search_bn = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50, cv=3,
                                   random_state=42, verbose=2)
rnd_search_bn.fit(X_train1, y_train1, X_valid=X_valid1, y_valid=y_valid1, n_epochs=1000)

# If you have Scikit-Learn 0.18 or earlier, you should upgrade, or use the fit_params argument:
# fit_params = dict(X_valid=X_valid1, y_valid=y_valid1, n_epochs=1000)
# rnd_search_bn = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
#                                    fit_params=fit_params, random_state=42, verbose=2)
# rnd_search_bn.fit(X_train1, y_train1)


Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] n_neurons=70, learning_rate=0.01, batch_size=50, batch_norm_momentum=0.99, activation=<function relu at 0x00000130278C82F0> 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0	Validation loss: 0.109807	Best loss: 0.109807	Accuracy: 97.50%
1	Validation loss: 0.075441	Best loss: 0.075441	Accuracy: 98.08%
2	Validation loss: 0.094208	Best loss: 0.075441	Accuracy: 97.77%
3	Validation loss: 0.068134	Best loss: 0.068134	Accuracy: 98.08%
4	Validation loss: 0.056180	Best loss: 0.056180	Accuracy: 98.20%
5	Validation loss: 0.041651	Best loss: 0.041651	Accuracy: 98.94%
6	Validation loss: 0.042806	Best loss: 0.041651	Accuracy: 98.71%
7	Validation loss: 0.051499	Best loss: 0.041651	Accuracy: 98.67%
8	Validation loss: 0.041128	Best loss: 0.041128	Accuracy: 98.79%
9	Validation loss: 0.049314	Best loss: 0.041128	Accuracy: 98.71%
10	Validation loss: 0.048240	Best loss: 0.041128	Accuracy: 98.63%
11	Validation loss: 0.051138	Best loss: 0.041128	Accuracy: 98.91%
12	Validation loss: 0.045838	Best loss: 0.041128	Accuracy: 99.02%
13	Validation loss: 0.039577	Best loss: 0.039577	Accuracy: 98.94%
14	Validation loss: 0.052058	Best loss: 0.039577	Accuracy: 98.71%
15	Validation loss: 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s


0	Validation loss: 0.098469	Best loss: 0.098469	Accuracy: 97.34%
1	Validation loss: 0.072711	Best loss: 0.072711	Accuracy: 98.20%
2	Validation loss: 0.059985	Best loss: 0.059985	Accuracy: 98.48%
3	Validation loss: 0.042210	Best loss: 0.042210	Accuracy: 98.63%
4	Validation loss: 0.066488	Best loss: 0.042210	Accuracy: 98.32%
5	Validation loss: 0.049259	Best loss: 0.042210	Accuracy: 98.51%
6	Validation loss: 0.049017	Best loss: 0.042210	Accuracy: 98.63%
7	Validation loss: 0.049211	Best loss: 0.042210	Accuracy: 98.55%
8	Validation loss: 0.048343	Best loss: 0.042210	Accuracy: 98.83%
9	Validation loss: 0.058696	Best loss: 0.042210	Accuracy: 98.75%
10	Validation loss: 0.048273	Best loss: 0.042210	Accuracy: 98.67%
11	Validation loss: 0.075906	Best loss: 0.042210	Accuracy: 98.44%
12	Validation loss: 0.055642	Best loss: 0.042210	Accuracy: 98.75%
13	Validation loss: 0.038560	Best loss: 0.038560	Accuracy: 98.98%
14	Validation loss: 0.041075	Best loss: 0.038560	Accuracy: 99.02%
15	Validation loss: 

...

1	Validation loss: 0.195577	Best loss: 0.195577	Accuracy: 97.42%
2	Validation loss: 0.130967	Best loss: 0.130967	Accuracy: 97.50%
3	Validation loss: 0.099921	Best loss: 0.099921	Accuracy: 97.93%
4	Validation loss: 0.098255	Best loss: 0.098255	Accuracy: 98.24%
5	Validation loss: 0.059813	Best loss: 0.059813	Accuracy: 98.36%
6	Validation loss: 0.100221	Best loss: 0.059813	Accuracy: 97.77%
7	Validation loss: 0.085438	Best loss: 0.059813	Accuracy: 97.85%
8	Validation loss: 0.082723	Best loss: 0.059813	Accuracy: 98.12%
9	Validation loss: 0.103186	Best loss: 0.059813	Accuracy: 98.16%
10	Validation loss: 0.078404	Best loss: 0.059813	Accuracy: 98.01%
11	Validation loss: 0.072333	Best loss: 0.059813	Accuracy: 98.16%
12	Validation loss: 0.123232	Best loss: 0.059813	Accuracy: 98.01%
13	Validation loss: 0.094659	Best loss: 0.059813	Accuracy: 98.20%
14	Validation loss: 0.083922	Best loss: 0.059813	Accuracy: 98.12%
15	Validation loss: 0.075327	Best loss: 0.059813	Accuracy: 98.51%
16	Validation loss:

18	Validation loss: 0.297049	Best loss: 0.044380	Accuracy: 97.62%
19	Validation loss: 0.038916	Best loss: 0.038916	Accuracy: 99.22%
20	Validation loss: 0.042383	Best loss: 0.038916	Accuracy: 99.10%
21	Validation loss: 0.042944	Best loss: 0.038916	Accuracy: 98.94%
22	Validation loss: 0.077951	Best loss: 0.038916	Accuracy: 98.79%
23	Validation loss: 0.127577	Best loss: 0.038916	Accuracy: 98.63%
24	Validation loss: 0.094017	Best loss: 0.038916	Accuracy: 98.44%
25	Validation loss: 0.057584	Best loss: 0.038916	Accuracy: 99.02%
26	Validation loss: 0.052234	Best loss: 0.038916	Accuracy: 98.98%
27	Validation loss: 0.069631	Best loss: 0.038916	Accuracy: 98.91%
28	Validation loss: 0.144169	Best loss: 0.038916	Accuracy: 98.67%
29	Validation loss: 0.128707	Best loss: 0.038916	Accuracy: 98.63%
30	Validation loss: 0.077145	Best loss: 0.038916	Accuracy: 98.91%
31	Validation loss: 0.050081	Best loss: 0.038916	Accuracy: 99.02%
32	Validation loss: 0.052511	Best loss: 0.038916	Accuracy: 98.98%
33	Validat

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 402.4min finished


0	Validation loss: 0.084599	Best loss: 0.084599	Accuracy: 97.54%
1	Validation loss: 0.046368	Best loss: 0.046368	Accuracy: 98.51%
2	Validation loss: 0.050000	Best loss: 0.046368	Accuracy: 98.59%
3	Validation loss: 0.056068	Best loss: 0.046368	Accuracy: 98.24%
4	Validation loss: 0.037067	Best loss: 0.037067	Accuracy: 98.83%
5	Validation loss: 0.031454	Best loss: 0.031454	Accuracy: 98.94%
6	Validation loss: 0.042112	Best loss: 0.031454	Accuracy: 98.67%
7	Validation loss: 0.047219	Best loss: 0.031454	Accuracy: 98.71%
8	Validation loss: 0.030473	Best loss: 0.030473	Accuracy: 99.18%
9	Validation loss: 0.036730	Best loss: 0.030473	Accuracy: 99.10%
10	Validation loss: 0.035061	Best loss: 0.030473	Accuracy: 99.10%
11	Validation loss: 0.031104	Best loss: 0.030473	Accuracy: 99.06%
12	Validation loss: 0.032551	Best loss: 0.030473	Accuracy: 99.06%
13	Validation loss: 0.024269	Best loss: 0.024269	Accuracy: 99.18%
14	Validation loss: 0.026430	Best loss: 0.024269	Accuracy: 99.18%
15	Validation loss: 

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=DNNClassifier(activation=<function elu at 0x0000013027896D08>,
                                           batch_norm_momentum=None,
                                           batch_size=20, dropout_rate=None,
                                           initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x0000013041242CC0>,
                                           learning_rate=0.01,
                                           n_hidden_layers=5, n_neurons=100,
                                           optimizer_class=<class 'tensorf...
                                                       <function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000131B4EE07B8>,
                                                       <function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000131B4EE0730>],
                                        'batch_norm_momentum': [0.9, 0.95, 0.98,
    

In [21]:
rnd_search_bn.best_params_

{'n_neurons': 160,
 'learning_rate': 0.01,
 'batch_size': 10,
 'batch_norm_momentum': 0.98,
 'activation': <function tensorflow.python.ops.gen_nn_ops.relu(features, name=None)>}

In [22]:
y_pred = rnd_search_bn.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9943568787701887

In [23]:
y_pred = dnn_clf.predict(X_train1)
accuracy_score(y_train1, y_pred)

0.9970753976745845

In [24]:
dnn_clf_dropout = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=500, learning_rate=0.01,
                                n_neurons=90, random_state=42,
                                dropout_rate=0.5)
dnn_clf_dropout.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
0	Validation loss: 0.132864	Best loss: 0.132864	Accuracy: 96.64%
1	Validation loss: 0.094044	Best loss: 0.094044	Accuracy: 97.11%
2	Validation loss: 0.096225	Best loss: 0.094044	Accuracy: 97.65%
3	Validation loss: 0.092286	Best loss: 0.092286	Accuracy: 97.73%
4	Validation loss: 0.079111	Best loss: 0.079111	Accuracy: 97.93%
5	Validation loss: 0.077223	Best loss: 0.077223	Accuracy: 97.97%
6	Validation loss: 0.077235	Best loss: 0.077223	Accuracy: 97.77%
7	Validation loss: 0.074484	Best loss: 0.074484	Accuracy: 98.08%
8	Validation loss: 0.073005	Best loss: 0.073005	Accuracy: 98.12%
9	Validation loss: 0.069229	Best loss: 0.069229	Accuracy: 98.16%
10	Validation loss: 0.081081	Best loss: 0.069229	Accuracy: 97.85%
11	Validation loss: 0.071001	Best loss: 0.069229	Accuracy: 98.12%
12	Validation loss: 0.064855	Best loss: 0.

DNNClassifier(activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000131B863D950>,
              batch_norm_momentum=None, batch_size=500, dropout_rate=0.5,
              initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x0000013041242CC0>,
              learning_rate=0.01, n_hidden_layers=5, n_neurons=90,
              optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
              random_state=42)

In [25]:
y_pred = dnn_clf_dropout.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.987935395991438

尝试添加dropout，再来

In [26]:
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_neurons": [10, 30, 50, 70, 90, 100, 120, 140, 160],
    "batch_size": [10, 50, 100, 500],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    # you could also try exploring different numbers of hidden layers, different optimizers, etc.
    #"n_hidden_layers": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    #"optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
    "dropout_rate": [0.2, 0.3, 0.4, 0.5, 0.6],
}

rnd_search_dropout = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
                                        cv=3, random_state=42, verbose=2)
rnd_search_dropout.fit(X_train1, y_train1, X_valid=X_valid1, y_valid=y_valid1, n_epochs=1000)

# If you have Scikit-Learn 0.18 or earlier, you should upgrade, or use the fit_params argument:
# fit_params = dict(X_valid=X_valid1, y_valid=y_valid1, n_epochs=1000)
# rnd_search_dropout = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
#                                         fit_params=fit_params, random_state=42, verbose=2)
# rnd_search_dropout.fit(X_train1, y_train1)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] n_neurons=70, learning_rate=0.01, dropout_rate=0.5, batch_size=100, activation=<function relu at 0x00000130278C82F0> 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0	Validation loss: 0.212666	Best loss: 0.212666	Accuracy: 94.64%
1	Validation loss: 0.229251	Best loss: 0.212666	Accuracy: 95.00%
2	Validation loss: 0.224572	Best loss: 0.212666	Accuracy: 96.05%
3	Validation loss: 0.200710	Best loss: 0.200710	Accuracy: 95.82%
4	Validation loss: 0.200189	Best loss: 0.200189	Accuracy: 95.66%
5	Validation loss: 0.221951	Best loss: 0.200189	Accuracy: 96.01%
6	Validation loss: 0.171350	Best loss: 0.171350	Accuracy: 95.82%
7	Validation loss: 0.188912	Best loss: 0.171350	Accuracy: 95.47%
8	Validation loss: 0.208015	Best loss: 0.171350	Accuracy: 95.00%
9	Validation loss: 0.194022	Best loss: 0.171350	Accuracy: 95.35%
10	Validation loss: 0.176668	Best loss: 0.171350	Accuracy: 95.86%
11	Validation loss: 0.178301	Best loss: 0.171350	Accuracy: 96.13%
12	Validation loss: 0.166062	Best loss: 0.166062	Accuracy: 96.33%
13	Validation loss: 0.161129	Best loss: 0.161129	Accuracy: 95.70%
14	Validation loss: 0.178064	Best loss: 0.161129	Accuracy: 96.40%
15	Validation loss: 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   34.7s remaining:    0.0s


0	Validation loss: 0.231269	Best loss: 0.231269	Accuracy: 93.90%
1	Validation loss: 0.222216	Best loss: 0.222216	Accuracy: 95.47%
2	Validation loss: 0.149695	Best loss: 0.149695	Accuracy: 96.40%
3	Validation loss: 0.187961	Best loss: 0.149695	Accuracy: 95.43%
4	Validation loss: 0.162758	Best loss: 0.149695	Accuracy: 96.13%
5	Validation loss: 0.169980	Best loss: 0.149695	Accuracy: 96.01%
6	Validation loss: 0.177524	Best loss: 0.149695	Accuracy: 96.64%
7	Validation loss: 0.166314	Best loss: 0.149695	Accuracy: 96.36%
8	Validation loss: 0.135812	Best loss: 0.135812	Accuracy: 96.52%
9	Validation loss: 0.153806	Best loss: 0.135812	Accuracy: 96.48%
10	Validation loss: 0.167636	Best loss: 0.135812	Accuracy: 96.05%
11	Validation loss: 0.157043	Best loss: 0.135812	Accuracy: 96.44%
12	Validation loss: 0.151775	Best loss: 0.135812	Accuracy: 96.40%
13	Validation loss: 0.181892	Best loss: 0.135812	Accuracy: 96.40%
14	Validation loss: 0.142353	Best loss: 0.135812	Accuracy: 96.05%
15	Validation loss: 

...

30	Validation loss: 2948.037842	Best loss: 569.222046	Accuracy: 18.73%
31	Validation loss: 397.219513	Best loss: 397.219513	Accuracy: 22.01%
32	Validation loss: 629.304382	Best loss: 397.219513	Accuracy: 20.91%
33	Validation loss: 538.449463	Best loss: 397.219513	Accuracy: 19.08%
34	Validation loss: 186.655304	Best loss: 186.655304	Accuracy: 21.85%
35	Validation loss: 113.522812	Best loss: 113.522812	Accuracy: 31.47%
36	Validation loss: 431.579315	Best loss: 113.522812	Accuracy: 20.91%
37	Validation loss: 886.284607	Best loss: 113.522812	Accuracy: 22.01%
38	Validation loss: 243.414749	Best loss: 113.522812	Accuracy: 25.92%
39	Validation loss: 295.605927	Best loss: 113.522812	Accuracy: 33.03%
40	Validation loss: 177.889801	Best loss: 113.522812	Accuracy: 22.01%
41	Validation loss: 256.066956	Best loss: 113.522812	Accuracy: 28.19%
42	Validation loss: 555.409668	Best loss: 113.522812	Accuracy: 22.01%
43	Validation loss: 33.086132	Best loss: 33.086132	Accuracy: 31.12%
44	Validation loss: 8

1	Validation loss: 1.897180	Best loss: 1.764949	Accuracy: 19.08%
2	Validation loss: 1.766144	Best loss: 1.764949	Accuracy: 19.27%
3	Validation loss: 1.750370	Best loss: 1.750370	Accuracy: 22.01%
4	Validation loss: 1.630450	Best loss: 1.630450	Accuracy: 20.91%
5	Validation loss: 1.642994	Best loss: 1.630450	Accuracy: 19.08%
6	Validation loss: 1.743618	Best loss: 1.630450	Accuracy: 19.27%
7	Validation loss: 1.874810	Best loss: 1.630450	Accuracy: 18.73%
8	Validation loss: 1.779562	Best loss: 1.630450	Accuracy: 18.73%
9	Validation loss: 1.959646	Best loss: 1.630450	Accuracy: 18.73%
10	Validation loss: 1.708315	Best loss: 1.630450	Accuracy: 22.01%
11	Validation loss: 1.705815	Best loss: 1.630450	Accuracy: 22.01%
12	Validation loss: 1.644244	Best loss: 1.630450	Accuracy: 20.91%
13	Validation loss: 1.652501	Best loss: 1.630450	Accuracy: 19.08%
14	Validation loss: 1.829212	Best loss: 1.630450	Accuracy: 19.27%
15	Validation loss: 1.655408	Best loss: 1.630450	Accuracy: 19.08%
16	Validation loss:

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 103.0min finished


0	Validation loss: 0.081764	Best loss: 0.081764	Accuracy: 97.81%
1	Validation loss: 0.093691	Best loss: 0.081764	Accuracy: 97.77%
2	Validation loss: 0.057384	Best loss: 0.057384	Accuracy: 98.71%
3	Validation loss: 0.059743	Best loss: 0.057384	Accuracy: 98.32%
4	Validation loss: 0.091097	Best loss: 0.057384	Accuracy: 97.93%
5	Validation loss: 0.051933	Best loss: 0.051933	Accuracy: 98.75%
6	Validation loss: 0.057568	Best loss: 0.051933	Accuracy: 98.55%
7	Validation loss: 0.056405	Best loss: 0.051933	Accuracy: 98.71%
8	Validation loss: 0.055498	Best loss: 0.051933	Accuracy: 98.48%
9	Validation loss: 0.053901	Best loss: 0.051933	Accuracy: 98.51%
10	Validation loss: 0.055463	Best loss: 0.051933	Accuracy: 98.59%
11	Validation loss: 0.052690	Best loss: 0.051933	Accuracy: 98.59%
12	Validation loss: 0.078396	Best loss: 0.051933	Accuracy: 98.08%
13	Validation loss: 0.058837	Best loss: 0.051933	Accuracy: 98.40%
14	Validation loss: 0.058450	Best loss: 0.051933	Accuracy: 98.44%
15	Validation loss: 

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=DNNClassifier(activation=<function elu at 0x0000013027896D08>,
                                           batch_norm_momentum=None,
                                           batch_size=20, dropout_rate=None,
                                           initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x0000013041242CC0>,
                                           learning_rate=0.01,
                                           n_hidden_layers=5, n_neurons=100,
                                           optimizer_class=<class 'tensorf...
                                                       <function leaky_relu.<locals>.parametrized_leaky_relu at 0x000001300B4A0840>,
                                                       <function leaky_relu.<locals>.parametrized_leaky_relu at 0x000001300DA6DA60>],
                                        'batch_size': [10, 50, 100, 500],
           

In [27]:
rnd_search_dropout.best_params_

{'n_neurons': 70,
 'learning_rate': 0.01,
 'dropout_rate': 0.2,
 'batch_size': 100,
 'activation': <function __main__.leaky_relu.<locals>.parametrized_leaky_relu(z, name=None)>}

In [28]:
y_pred = rnd_search_dropout.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9896867094765519

## 9.迁移学习

重用预训练的隐藏层和之前的模型，冻结的同时，重新训练softmax

In [29]:
reset_graph()

restore_saver = tf.train.import_meta_graph("./my_best_mnist_model_0_to_4.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
loss = tf.get_default_graph().get_tensor_by_name("loss:0")
Y_proba = tf.get_default_graph().get_tensor_by_name("Y_proba:0")
logits = Y_proba.op.inputs[0]
accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")

In [30]:
learning_rate = 0.01
#  只训练softmax层
output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

In [31]:
# top1
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
five_frozen_saver = tf.train.Saver()

In [32]:
X_train2_full = X_train[y_train >= 5]
y_train2_full = y_train[y_train >= 5] - 5
X_valid2_full = X_valid[y_valid >= 5]
y_valid2_full = y_valid[y_valid >= 5] - 5
X_test2 = X_test[y_test >= 5]
y_test2 = y_test[y_test >= 5] - 5

只取100个来训练

In [33]:
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

In [34]:
X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

In [35]:
import time

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_best_mnist_model_0_to_4")
    t0 = time.time()
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_best_mnist_model_0_to_4
0	Validation loss: 1.203592	Best loss: 1.203592	Accuracy: 60.00%
1	Validation loss: 1.159314	Best loss: 1.159314	Accuracy: 59.33%
2	Validation loss: 1.119128	Best loss: 1.119128	Accuracy: 55.33%
3	Validation loss: 1.061447	Best loss: 1.061447	Accuracy: 62.67%
4	Validation loss: 1.024384	Best loss: 1.024384	Accuracy: 65.33%
5	Validation loss: 1.014763	Best loss: 1.014763	Accuracy: 64.67%
6	Validation loss: 0.996666	Best loss: 0.996666	Accuracy: 62.00%
7	Validation loss: 1.029045	Best loss: 0.996666	Accuracy: 64.67%
8	Validation loss: 0.986451	Best loss: 0.986451	Accuracy: 67.33%
9	Validation loss: 0.976389	Best loss: 0.976389	Accuracy: 66.67%
10	Validation loss: 0.984188	Best loss: 0.976389	Accuracy: 65.33%
11	Validation loss: 1.024522	Best loss: 0.976389	Accuracy: 66.00%
12	Validation loss: 1.062505	Best loss: 0.976389	Accuracy: 60.67%
13	Validation loss: 0.995879	Best loss: 0.976389	Accuracy: 68.00%
14	Validation l

缓冲冻结层，来训练

In [36]:
hidden5_out = tf.get_default_graph().get_tensor_by_name("hidden5_out:0")

In [37]:
import time

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_best_mnist_model_0_to_4")
    t0 = time.time()
    # 缓冲冻结层
    hidden5_train = hidden5_out.eval(feed_dict={X: X_train2, y: y_train2})
    hidden5_valid = hidden5_out.eval(feed_dict={X: X_valid2, y: y_valid2})
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            h5_batch, y_batch = hidden5_train[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={hidden5_out: h5_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={hidden5_out: hidden5_valid, y: y_valid2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_best_mnist_model_0_to_4
0	Validation loss: 1.263521	Best loss: 1.263521	Accuracy: 54.00%
1	Validation loss: 1.180370	Best loss: 1.180370	Accuracy: 59.33%
2	Validation loss: 1.082855	Best loss: 1.082855	Accuracy: 65.33%
3	Validation loss: 1.044809	Best loss: 1.044809	Accuracy: 60.67%
4	Validation loss: 1.022364	Best loss: 1.022364	Accuracy: 65.33%
5	Validation loss: 1.004187	Best loss: 1.004187	Accuracy: 67.33%
6	Validation loss: 0.995081	Best loss: 0.995081	Accuracy: 64.00%
7	Validation loss: 1.010094	Best loss: 0.995081	Accuracy: 67.33%
8	Validation loss: 0.989254	Best loss: 0.989254	Accuracy: 68.67%
9	Validation loss: 1.077037	Best loss: 0.989254	Accuracy: 60.00%
10	Validation loss: 1.067207	Best loss: 0.989254	Accuracy: 60.00%
11	Validation loss: 1.003286	Best loss: 0.989254	Accuracy: 63.33%
12	Validation loss: 0.945468	Best loss: 0.945468	Accuracy: 69.33%
13	Validation loss: 1.012329	Best loss: 0.945468	Accuracy: 67.33%
14	Validation l

In [38]:
reset_graph()

n_outputs = 5

restore_saver = tf.train.import_meta_graph("./my_best_mnist_model_0_to_4.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
# 去掉第五层 从第四层过softmax
hidden4_out = tf.get_default_graph().get_tensor_by_name("hidden4_out:0")
logits = tf.layers.dense(hidden4_out, n_outputs, kernel_initializer=he_init, name="new_logits")

Y_proba = tf.nn.softmax(logits)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [39]:
learning_rate = 0.01
# 只训练softmax
output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="new_logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

init = tf.global_variables_initializer()
four_frozen_saver = tf.train.Saver()

In [40]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_best_mnist_model_0_to_4")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = four_frozen_saver.save(sess, "./my_mnist_model_5_to_9_four_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    four_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_four_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_best_mnist_model_0_to_4
0	Validation loss: 1.037749	Best loss: 1.037749	Accuracy: 59.33%
1	Validation loss: 1.016575	Best loss: 1.016575	Accuracy: 66.00%
2	Validation loss: 1.040566	Best loss: 1.016575	Accuracy: 60.00%
3	Validation loss: 0.981040	Best loss: 0.981040	Accuracy: 64.00%
4	Validation loss: 0.970343	Best loss: 0.970343	Accuracy: 69.33%
5	Validation loss: 0.944685	Best loss: 0.944685	Accuracy: 70.00%
6	Validation loss: 0.934863	Best loss: 0.934863	Accuracy: 70.67%
7	Validation loss: 0.936754	Best loss: 0.934863	Accuracy: 71.33%
8	Validation loss: 0.905589	Best loss: 0.905589	Accuracy: 70.67%
9	Validation loss: 0.909335	Best loss: 0.905589	Accuracy: 72.00%
10	Validation loss: 0.914879	Best loss: 0.905589	Accuracy: 70.00%
11	Validation loss: 0.933296	Best loss: 0.905589	Accuracy: 70.00%
12	Validation loss: 0.929592	Best loss: 0.905589	Accuracy: 66.00%
13	Validation loss: 0.906715	Best loss: 0.905589	Accuracy: 73.33%
14	Validation l

解冻前两层和softmax

In [41]:
learning_rate = 0.01

unfrozen_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[34]|new_logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam3")
training_op = optimizer.minimize(loss, var_list=unfrozen_vars)

init = tf.global_variables_initializer()
two_frozen_saver = tf.train.Saver()

In [42]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    four_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_four_frozen")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = two_frozen_saver.save(sess, "./my_mnist_model_5_to_9_two_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    two_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_two_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_mnist_model_5_to_9_four_frozen
0	Validation loss: 1.230271	Best loss: 1.230271	Accuracy: 68.00%
1	Validation loss: 0.707271	Best loss: 0.707271	Accuracy: 84.00%
2	Validation loss: 0.640864	Best loss: 0.640864	Accuracy: 81.33%
3	Validation loss: 0.727703	Best loss: 0.640864	Accuracy: 79.33%
4	Validation loss: 0.773654	Best loss: 0.640864	Accuracy: 80.67%
5	Validation loss: 0.782815	Best loss: 0.640864	Accuracy: 84.00%
6	Validation loss: 0.934228	Best loss: 0.640864	Accuracy: 81.33%
7	Validation loss: 1.053239	Best loss: 0.640864	Accuracy: 77.33%
8	Validation loss: 0.792953	Best loss: 0.640864	Accuracy: 82.00%
9	Validation loss: 0.728003	Best loss: 0.640864	Accuracy: 84.67%
10	Validation loss: 0.929337	Best loss: 0.640864	Accuracy: 84.67%
11	Validation loss: 0.878843	Best loss: 0.640864	Accuracy: 85.33%
12	Validation loss: 0.891040	Best loss: 0.640864	Accuracy: 81.33%
13	Validation loss: 1.112440	Best loss: 0.640864	Accuracy: 84.00%
14	Valid

全部解冻

In [43]:
learning_rate = 0.01

optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam4")
# 从模型图获取的交叉熵损失
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
no_frozen_saver = tf.train.Saver()

In [44]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    two_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_two_frozen")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = no_frozen_saver.save(sess, "./my_mnist_model_5_to_9_no_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    no_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_no_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_mnist_model_5_to_9_two_frozen
0	Validation loss: 0.477066	Best loss: 0.477066	Accuracy: 83.33%
1	Validation loss: 0.794025	Best loss: 0.477066	Accuracy: 87.33%
2	Validation loss: 0.722156	Best loss: 0.477066	Accuracy: 91.33%
3	Validation loss: 2.446293	Best loss: 0.477066	Accuracy: 82.67%
4	Validation loss: 0.846944	Best loss: 0.477066	Accuracy: 89.33%
5	Validation loss: 1.189460	Best loss: 0.477066	Accuracy: 90.00%
6	Validation loss: 0.924417	Best loss: 0.477066	Accuracy: 90.67%
7	Validation loss: 1.254551	Best loss: 0.477066	Accuracy: 86.67%
8	Validation loss: 1.013173	Best loss: 0.477066	Accuracy: 90.00%
9	Validation loss: 1.699855	Best loss: 0.477066	Accuracy: 92.00%
10	Validation loss: 1.875251	Best loss: 0.477066	Accuracy: 89.33%
11	Validation loss: 1.126102	Best loss: 0.477066	Accuracy: 80.67%
12	Validation loss: 0.684392	Best loss: 0.477066	Accuracy: 92.00%
13	Validation loss: 0.901931	Best loss: 0.477066	Accuracy: 88.00%
14	Valida

小样本重新训练的结果

In [45]:
dnn_clf_5_to_9 = DNNClassifier(n_hidden_layers=4, random_state=42)
dnn_clf_5_to_9.fit(X_train2, y_train2, n_epochs=1000, X_valid=X_valid2, y_valid=y_valid2)

0	Validation loss: 0.674605	Best loss: 0.674605	Accuracy: 80.67%
1	Validation loss: 0.584848	Best loss: 0.584848	Accuracy: 88.67%
2	Validation loss: 0.647304	Best loss: 0.584848	Accuracy: 84.00%
3	Validation loss: 0.530457	Best loss: 0.530457	Accuracy: 87.33%
4	Validation loss: 0.684249	Best loss: 0.530457	Accuracy: 90.67%
5	Validation loss: 0.532007	Best loss: 0.530457	Accuracy: 89.33%
6	Validation loss: 0.718040	Best loss: 0.530457	Accuracy: 90.00%
7	Validation loss: 0.922250	Best loss: 0.530457	Accuracy: 84.67%
8	Validation loss: 1.090463	Best loss: 0.530457	Accuracy: 87.33%
9	Validation loss: 1.304759	Best loss: 0.530457	Accuracy: 90.67%
10	Validation loss: 0.723792	Best loss: 0.530457	Accuracy: 92.00%
11	Validation loss: 2.214152	Best loss: 0.530457	Accuracy: 84.67%
12	Validation loss: 0.852909	Best loss: 0.530457	Accuracy: 90.67%
13	Validation loss: 1.488704	Best loss: 0.530457	Accuracy: 90.67%
14	Validation loss: 1.755688	Best loss: 0.530457	Accuracy: 90.00%
15	Validation loss: 

DNNClassifier(activation=<function elu at 0x0000013027896D08>,
              batch_norm_momentum=None, batch_size=20, dropout_rate=None,
              initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x0000013041242CC0>,
              learning_rate=0.01, n_hidden_layers=4, n_neurons=100,
              optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
              random_state=42)

In [46]:
y_pred = dnn_clf_5_to_9.predict(X_test2)
accuracy_score(y_test2, y_pred)

0.8481793869574161

# 10.在辅助任务上预训练

构建A和B，100个神经元+5层+He初始化+ELU激活函数

In [47]:
n_inputs = 28 * 28 # MNIST

reset_graph()

X = tf.placeholder(tf.float32, shape=(None, 2, n_inputs), name="X")
X1, X2 = tf.unstack(X, axis=1)

In [48]:
y = tf.placeholder(tf.int32, shape=[None, 1])

In [ ]:
# he_init = tf.variance_scaling_initializer()

# def dnn(inputs, n_hidden_layers=5, n_neurons=100, name=None,
#         activation=tf.nn.elu, initializer=he_init):
#     with tf.variable_scope(name, "dnn"):
#         for layer in range(n_hidden_layers):
#             inputs = tf.layers.dense(inputs, n_neurons, activation=activation,
#                                      kernel_initializer=initializer,
#                                      name="hidden%d" % (layer + 1))
#         return inputs

In [49]:
# A和B
dnn1 = dnn(X1, name="DNN_A")
dnn2 = dnn(X2, name="DNN_B")

In [50]:
# 水平连接
dnn_outputs = tf.concat([dnn1, dnn2], axis=1)

In [51]:
dnn1.shape

TensorShape([Dimension(None), Dimension(100)])

In [52]:
dnn2.shape

TensorShape([Dimension(None), Dimension(100)])

In [53]:
dnn_outputs.shape

TensorShape([Dimension(None), Dimension(200)])

In [54]:
# 添加输出层
hidden = tf.layers.dense(dnn_outputs, units=10, activation=tf.nn.elu, kernel_initializer=he_init)
logits = tf.layers.dense(hidden, units=1, kernel_initializer=he_init)
y_proba = tf.nn.sigmoid(logits)

简单的二分类任务开始，<0 是 >0否

In [55]:
y_pred = tf.cast(tf.greater_equal(logits, 0), tf.int32)

计算损失

In [56]:
y_as_float = tf.cast(y, tf.float32)
xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_as_float, logits=logits)
loss = tf.reduce_mean(xentropy)

In [57]:
learning_rate = 0.01
momentum = 0.95

optimizer = tf.train.MomentumOptimizer(learning_rate, momentum, use_nesterov=True)
training_op = optimizer.minimize(loss)

In [58]:
# 计算指标
y_pred_correct = tf.equal(y_pred, y)
accuracy = tf.reduce_mean(tf.cast(y_pred_correct, tf.float32))

In [59]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [60]:
# 切分数据集
X_train1 = X_train
y_train1 = y_train

X_train2 = X_valid
y_train2 = y_valid

X_test = X_test
y_test = y_test

In [61]:
# 产生批次以及标注结果  如果是同一个数字为1，不是为0
def generate_batch(images, labels, batch_size):
    size1 = batch_size // 2
    size2 = batch_size - size1
    if size1 != size2 and np.random.rand() > 0.5:
        size1, size2 = size2, size1
    X = []
    y = []
    while len(X) < size1:
        rnd_idx1, rnd_idx2 = np.random.randint(0, len(images), 2)
        if rnd_idx1 != rnd_idx2 and labels[rnd_idx1] == labels[rnd_idx2]:
            X.append(np.array([images[rnd_idx1], images[rnd_idx2]]))
            y.append([1])
    while len(X) < batch_size:
        rnd_idx1, rnd_idx2 = np.random.randint(0, len(images), 2)
        if labels[rnd_idx1] != labels[rnd_idx2]:
            X.append(np.array([images[rnd_idx1], images[rnd_idx2]]))
            y.append([0])
    rnd_indices = np.random.permutation(batch_size)
    return np.array(X)[rnd_indices], np.array(y)[rnd_indices]

查看结果

In [62]:
batch_size = 5
X_batch, y_batch = generate_batch(X_train1, y_train1, batch_size)

In [63]:
X_batch.shape, X_batch.dtype

((5, 2, 784), dtype('float32'))

In [64]:
plt.figure(figsize=(3, 3 * batch_size))
plt.subplot(121)
plt.imshow(X_batch[:,0].reshape(28 * batch_size, 28), cmap="binary", interpolation="nearest")
plt.axis('off')
plt.subplot(122)
plt.imshow(X_batch[:,1].reshape(28 * batch_size, 28), cmap="binary", interpolation="nearest")
plt.axis('off')
plt.show()

In [65]:
y_batch

array([[1],
       [0],
       [0],
       [1],
       [0]])

In [66]:
X_test1, y_test1 = generate_batch(X_test, y_test, batch_size=len(X_test))

开始训练

In [67]:
n_epochs = 100
batch_size = 500

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(len(X_train1) // batch_size):
            X_batch, y_batch = generate_batch(X_train1, y_train1, batch_size)
            loss_val, _ = sess.run([loss, training_op], feed_dict={X: X_batch, y: y_batch})
        print(epoch, "Train loss:", loss_val)
        if epoch % 5 == 0:
            acc_test = accuracy.eval(feed_dict={X: X_test1, y: y_test1})
            print(epoch, "Test accuracy:", acc_test)

    save_path = saver.save(sess, "./my_digit_comparison_model.ckpt")

0 Train loss: 0.6910327
0 Test accuracy: 0.542
1 Train loss: 0.6035354
2 Train loss: 0.54946035
3 Train loss: 0.47047248
4 Train loss: 0.4060757
5 Train loss: 0.38308156
5 Test accuracy: 0.824
6 Train loss: 0.3904727
7 Train loss: 0.3390794
8 Train loss: 0.32106707
9 Train loss: 0.31792688
10 Train loss: 0.2449429
10 Test accuracy: 0.8881
11 Train loss: 0.29292354
12 Train loss: 0.23225448
13 Train loss: 0.23180932
14 Train loss: 0.19877928
15 Train loss: 0.2006546
15 Test accuracy: 0.9203
16 Train loss: 0.19700494
17 Train loss: 0.18893136
18 Train loss: 0.1996545
19 Train loss: 0.24071646
20 Train loss: 0.18882026
20 Test accuracy: 0.9367
21 Train loss: 0.124191955
22 Train loss: 0.14013414
23 Train loss: 0.120789446
24 Train loss: 0.15721141
25 Train loss: 0.115078606
25 Test accuracy: 0.948
26 Train loss: 0.13891111
27 Train loss: 0.15260816
28 Train loss: 0.12343675
29 Train loss: 0.11543136
30 Train loss: 0.11402824
30 Test accuracy: 0.9507
31 Train loss: 0.11897089
32 Train loss

重用和冻结隐藏层，然后添加softmax

In [68]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")
# 冻结
dnn_outputs = dnn(X, name="DNN_A")
frozen_outputs = tf.stop_gradient(dnn_outputs)

logits = tf.layers.dense(frozen_outputs, n_outputs, kernel_initializer=he_init)
Y_proba = tf.nn.softmax(logits)

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

optimizer = tf.train.MomentumOptimizer(learning_rate, momentum, use_nesterov=True)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

dnn_A_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="DNN_A")
restore_saver = tf.train.Saver(var_list={var.op.name: var for var in dnn_A_vars})
saver = tf.train.Saver()

只在5000的样本上训练

In [69]:
n_epochs = 100
batch_size = 50

with tf.Session() as sess:
    init.run()
    # 读取模型参数
    restore_saver.restore(sess, "./my_digit_comparison_model.ckpt")

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 10 == 0:
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
            print(epoch, "Test accuracy:", acc_test)

    save_path = saver.save(sess, "./my_mnist_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_digit_comparison_model.ckpt
0 Test accuracy: 0.9724
10 Test accuracy: 0.9743
20 Test accuracy: 0.9738
30 Test accuracy: 0.9726
40 Test accuracy: 0.9734
50 Test accuracy: 0.9728
60 Test accuracy: 0.973
70 Test accuracy: 0.973
80 Test accuracy: 0.9726
90 Test accuracy: 0.9732


重新训练

In [70]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

dnn_outputs = dnn(X, name="DNN_A")

logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init)
Y_proba = tf.nn.softmax(logits)

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

optimizer = tf.train.MomentumOptimizer(learning_rate, momentum, use_nesterov=True)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

dnn_A_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="DNN_A")
restore_saver = tf.train.Saver(var_list={var.op.name: var for var in dnn_A_vars})
saver = tf.train.Saver()

In [71]:
n_epochs = 150
batch_size = 50

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 10 == 0:
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
            print(epoch, "Test accuracy:", acc_test)

    save_path = saver.save(sess, "./my_mnist_model_final.ckpt")

0 Test accuracy: 0.8694
10 Test accuracy: 0.9276
20 Test accuracy: 0.9366
30 Test accuracy: 0.9404
40 Test accuracy: 0.9466
50 Test accuracy: 0.9467
60 Test accuracy: 0.9465
70 Test accuracy: 0.9463
80 Test accuracy: 0.9462
90 Test accuracy: 0.9461
100 Test accuracy: 0.946
110 Test accuracy: 0.946
120 Test accuracy: 0.946
130 Test accuracy: 0.9461
140 Test accuracy: 0.9462
